In [2]:
archivo = "manfred2025-02-14 12:00:29.csv"

In [3]:
archivo

'manfred2025-02-14 12:00:29.csv'

In [55]:
import pandas as pd
import numpy as np
import emoji
import datetime


def limpieza_manfred(archivo):

    df = pd.read_csv(archivo)

    #Crea columna id y lo lleva a la primera posición
    df["id"] = ["id_man_" + str(i) for i in range(1, len(df) + 1)]
    columna_extraida_ = df.pop("id")
    df.insert(0, "id", columna_extraida_)
    
    #Nueva columna con el nombre de la empresa a partir de la url y la mueve a la segunda posición
    df["empresa"] = df["url"].apply(lambda x: x.split("/")[-1].split("-")[0])
    columna_extraida = df.pop("empresa")
    df.insert(1, "empresa", columna_extraida)
    
    
    #Elimina "tecnologias_Tecnologías_" de todos los nomnbres de columna
    df.columns = df.columns.str.replace("tecnologias_Tecnologías_", "", regex=False)
    
    #Renombra columnas que empiezan por detalles_1_, detalles_2_
    
    df = df.rename(columns={"detalles_1_SALARIO" : "Salario", "detalles_1_PRESENCIAL" : "Presencial", "detalles_1_TELETRABAJO" : "Teletrabajo", "detalles_1_REMOTO" : "Remoto", "detalles_2_DÍA LABORABLE" : "dia_laborable", "detalles_2_VACACIONES" : "vacaciones", "detalles_2_JORNADA LABORAL" : "jornada_laboral", "detalles_2_TURNO CONTINUO" : "turno_continuo", "detalles_1_VARIABLE" : "variable"})
    
    #Quitar " DÍAS" en columna vacaciones
    ###Pendiente revisar ILIMITADO
    
    df["vacaciones"] = df["vacaciones"].str.replace(" DÍAS", "")
    df["vacaciones"].value_counts()
    # Reemplazar "ILIMITADO" con NaN
    df.replace("ILIMITADO", np.nan, inplace=True)
    
    #Columna habilidades_Otras habilidades. 
    
    # Convierte los strings a listas.
    import ast
    df["habilidades_Otras habilidades"] = df["habilidades_Otras habilidades"].apply(ast.literal_eval)
    
    #Crea una lista con las habilidades únicas
    todas_habilidades = sum(df["habilidades_Otras habilidades"], [])
    habilidades_unicas = list(set(todas_habilidades))
    
    # Crea una columna para cada habilidad única y usa la columna original para poblar las nuevas conlumnas con 1 o 0
    for habilidad in habilidades_unicas:
        df[habilidad] = df["habilidades_Otras habilidades"].apply(lambda x: 1 if habilidad in x else 0)
    
    #Elimina la columna habilidades_Otras habilidades. 
    df = df.drop("habilidades_Otras habilidades", axis = 1)
    
    #Dividimos las columnas en 3 categorías y generamos un df para cada uno de ellos.
    programas = ["id", "Python", "C++", "Cinema 4D", "CRM", "Canva", "Trello", "Google Analytics", "Adobe Photoshop", 
        "Adobe Illustrator", "WordPress", "Java", "SQL", "SOAP", "OpenApi", "JavaScript", "Cobol", 
        "C#", ".NET", "JSON", "Angular2+", "Microsoft Excel", "Hubspot", "AWS", "Kafka", "Argo CD", 
        "Flux", "React", "Kotlin", "Swift", "PostgreSQL", "Spring", "Azure", "Kubernetes", "Terraform", 
        "Docker", "Linux", "Git", "Golang", "Rust", "GitLab", "Django", "Celery", "Fastapi", "MySQL", 
        "NodeJS", "TypeScript", "MVC", "dbt", "Google Cloud", "Airflow", "z/OS", "DB2", "PowerShell", 
        "Bash", "Magento", "Prestashop", "BigCommerce", "Salesforce", "Shopify", "WooCommerce", 
        "RabbitMQ", "NextJS", "Tailwind", "Jenkins", "Databricks", "Prometheus", "Google Sheets", "nestJS", 
        "Vue", "MQTT", "Express", "Koa", "Spring Boot", "Go", "Bootstrap", "Pytest", "Github", "PHP", 
        "Symfony", "Figma", "HTML", "CSS", "Jest", "Hibernate", "API", "WebSockets", "Svelte", "BigQuery", 
        "Redux", "Ansible", "SQLServer", "Visual Studio", "REST", "ETL", "S3", "Redshift", "Snowflake", 
        "Auth0", "Tableau", "IIS", "Pulumi", "Azure DevOps", "Laravel", "Yii", "AWS ECS", "Nest", "PySpark", 
        "AWS Lambda", "Selenium", "Appium", "Cypress", "RxJS", "Clojure", "Grafana", "ELK", "Spark", "Jmeter", 
        "Gatling", "Apple iOS", "SwiftUI", "XCTest", "UIKit", "Atlassian", "Groovy", "ServiceNow", "Ethereum", 
        "Web3", ".Net Core", "Swagger", "Helm", "Datadog", "Kong", "Quarkus", "Vitest", "TestingLibrary", 
        "Metabase", "Webpack", "Elasticsearch", "Dart", "Flutter", "Blender", "Ruby on Rails", "Citrix", 
        "Windows", "VMWare", "Hyper-v", "Xen", "R", "Three.js", "MongoDB", "React Native", "Jira", "NoSQL", 
        "Scrapy", "Apache Beam", "Nagios", "Zabbix", "DNS", "HTTP4s", "Razor", "Shiny", "Xray", "AWS Cloudwatch", 
        "Google Ads", "Pipedrive", "Notion", "Monday", "DialogFlow", "GraphQL", "jQuery", "AngularJS", "Ionic", 
        "Viper", "Android", "Slack", "Mocha", "junit4", "Amplitude", "Mixpanel", "DynamoDB", "SaaS", "Redis", 
        "KVM", "Solr", "Flask", "Sklearn", "Google Looker Studio", "Scala", "Gitflow", "ASP.net", "Pandas", 
        "Numpy", "Google Firebase", "Riverpod", "gRPC", "Protobuf", "PHPunit", "Drupal", "Zoho", "Fortran", 
        "Vite", "TensorFlow", "ActiveMQ", "Cisco", "Microsoft Sharepoint", "Microsoft Dynamics 365", "Cassandra", 
        "Dagger2", "Core Data", "Ruby", "PyTorch", "AWS SageMaker", "Oracle", "Microsoft Power BI", 
        "Google Tag Manager", "Maven", "JUnit", "Blazor", "Microsoft", "javacript", "Sketch", "Adobe Xd", 
        "Axure", "InfluxDB", "Flink", "Postman", "JBoss", "Transact-SQL (T-SQL)", "PL/SQL", "Cordova", 
        "BlueTooth", "Nuxt.js", "Babel", "Microsoft Office 365", "Objective-C", "OpenCV", "Jasmine", 
        "Productboard", "Github Actions", "openlayers", "CARTO", "ArcGIS", "Leaflet", "Geoserver", "Google AdManager", 
        "Google Data Studio", "Google Optimize", "Couchbase", "Elixir", "Adobe Premiere Pro", "Cucumber", 
        "Sagas", "Apollo", "MVVM", "ClickHouse", "Micronaut Framework", "Airtable", "Asana", "Specflow", "Puppeteer", 
        "Miro", "Backbone", "CMS", "KendoUI", "Adobe Suit", "Apache", "Karate", "PACT", "MariaDB", "TestNg", 
        "AWS-User", "Apple macOS", "Microsoft Active Directory", "Storybook", "Keycloak", "slq-server", "GraalVM", 
        "FlexBox", "Unix", "data-lake", "shell-scripting", "postgres", "rabbit", "reactive-programming", "Puppet", 
        "Unity3D", "SAP", "Nginx", "Invision", "Chef", "Hotjar", "LookBack", "Maze", "Webflow", "CMake", "Qt", 
        "Alfresco", "MapBox", "ChartJS", "Google Maps", "OpenGL", "UML", "Genesys", "SIP", "Vuex", "VanillaJS", 
        "Hadoop", "OpenShift", "Hazelcast", "Android Studio", "PowerCenter", "Microsoft SQL Integration Services (SSIS)", 
        "Sass", "Ember", "Entity", "CodeIgniter", "OpenStack", "NgRx", "WPF", "FlaUI", "Gherkin", "Playwright", 
        "SonarQube", "Nexus", "Fortinet", "Vagrant", "F5", "Aruba", "Infoblox", "Palo Alto", "Zscaler", "MobileIron", 
        "PRTG", "VMware NSX", "WebComponents", "New Relic", "Confluence", "Plesk", "AJAX", "SCADA", "Kibana", 
        "Microstrategy", "BI", "Adobe Analytics", "Adobe Campaign", "Selligent", "screen-scraping", "TestFlight", 
        "Odoo", "WebApi", "RollupJS", "YAML", "Lucene", "RedHat", "Segment", "Facebook Ads", "Linkedin Ads", 
        "Bitbucket", "Lua", "Zend", "CircleCI", "Serverless", "Zapier", "Apache Tomcat", "git-bash"]
    
    habilidades = ["id", "Pensamiento creativo", "Tolerancia a la incertidumbre", "Resistencia a la frustración", 
        "Comunicación verbal", "Capacidad de escucha", "Capacidad de mentorización", "Capacidad de dar feedback", 
        "Adaptabilidad al cambio", "Visión estratégica", "Proactividad", "Capacidad de recibir feedback", 
        "Trabajo en equipo", "Habilidades de negociación", "Liderazgo", "Atención al detalle", 
        "Inteligencia emocional", "Capacidad de abstracción", "Capacidad de presentación", "Comunicación escrita", 
        "Pensamiento analítico", "Comunicación intercultural", "Autonomía en el aprendizaje", "Capacidad de autogestión", 
        "Aprendizaje Continuo", "Gestión de equipo", "Visión crítica"]
    
    general = ["id", "empresa", "url", "fecha_extraccion", "titulo", "oferta_activada", "Salario", "Presencial", 
        "Teletrabajo", "dia_laborable", "vacaciones", "jornada_laboral", "variable", "turno_continuo", "Remoto"]
    
    programas_df = df[programas]
    habilidades_df = df[habilidades]
    general_df = df[general]
    
    
    #Poblamos programas_df con boolenos en función de lo que la oferta solicita
    programas_df.loc[:,programas_df.columns != "id"] = programas_df.loc[:,programas_df.columns != "id"].fillna(0)
    programas_df.loc[:,programas_df.columns != "id"] = programas_df.loc[:,programas_df.columns != "id"].map(lambda x: 1 if x != 0 else 0)
    
    #Sacamos los emojis de la columna titulo
    general_df["titulo"] = general_df["titulo"].apply(lambda x: emoji.replace_emoji(x, replace="")) 
    
    #Modificamos la columna oferta activada a booleanos
    general_df["oferta_activada"] = general_df["oferta_activada"].apply(lambda x: 1 if x == "Activa" else 0)
    
    #Limpieza columna Salario
    general_df["Salario"] =  general_df["Salario"].str.replace("DESDE €", "").str.replace("HASTA €","").str.replace("€","").str.replace("K","").str.replace("£", "")
    general_df["Salario"] =  general_df["Salario"].str.replace("HASTA \u202f", "").str.replace("HASTA US$", "").str.replace("US$", "").str.replace("MXN$", "")
    general_df[["Salario_desde", "Salario_hasta"]] = general_df["Salario"].str.split("-", expand= True)
    general_df["Salario_hasta"] = general_df["Salario_hasta"].fillna(general_df["Salario_desde"])
    general_df[["Salario_desde", "Salario_hasta"]] = general_df[["Salario_desde", "Salario_hasta"]].astype("float")
    general_df.drop(columns = ["Salario"], inplace = True)
    
    #Limpieza columna turno_continuo  
    general_df["turno_continuo"] = general_df["turno_continuo"].apply(lambda x : "No" if pd.isna(x) else x)
    
    #Limpieza de variable y la lleva hasta la posición 7, al lado de sueldo
    general_df["variable"] = general_df["variable"].str.replace("+€","").str.replace("K","").str.replace("+\u202f€","").str.replace(",",".").astype("float")
    general_df["variable"] = general_df["variable"].fillna(0)
    columna_variable_extraida = general_df.pop("variable")
    general_df.insert(7, "variable", columna_variable_extraida)
    
    #Basandose en las columnas teletrabajo y Remoto actualizamos la columna teletrabajo con el porcentaje de teletrabajo
    #Que permite la oferta
    #Drop de la columna remoto
    
    general_df["Teletrabajo"] = general_df.apply(lambda x : "100%" if pd.isna(x ["Teletrabajo"]) and x["Remoto"] == "100%" else ("0" if pd.isna(x["Teletrabajo"]) and pd.isna(x["Remoto"]) else x["Teletrabajo"]), axis=1)
    general_df["Teletrabajo"] = general_df["Teletrabajo"].str.replace("%","").astype("int")
    general_df = general_df.drop("Remoto", axis = 1)
    
    #Cambiamos las vacaciones ilimitadas a nulas
    general_df["vacaciones"] = general_df["vacaciones"].replace("días ilimitados", np.nan)
    
    #Cambio de jornadas
    general_df["Jornada"] = np.where(
        (general_df["jornada_laboral"].isna()) & (general_df["dia_laborable"] == "JORNADA COMPLETA"), "JORNADA COMPLETA",
        np.where(
            (general_df["jornada_laboral"] == "FLEXIBLE") & (general_df["dia_laborable"] == "JORNADA COMPLETA"), "JORNADA COMPLETA FLEXIBLE",
                np.where(
                    (general_df["jornada_laboral"].isna()) & (general_df["dia_laborable"].isna()), "JORNADA COMPLETA",
                    "JORNADA COMPLETA")))
    
    general_df.drop(columns = ["jornada_laboral", "dia_laborable"], inplace = True)
    
    #Convertir turno continuo a booleano
    general_df["turno_continuo"] = np.where(general_df["turno_continuo"] == "No", 0, 1)
    
    # Crear un timestamp con la fecha y hora actual
    timestamp = datetime.datetime.now()
    
    general_df.to_csv(f"general_limpio.csv", index=False)
    habilidades_df.to_csv(f"habilidades_matricial.csv", index=False)
    programas_df.to_csv(f"programas_matricial.csv", index=False)

    return general_df, habilidades_df, programas_df

In [57]:
limpieza_manfred(archivo)

/tmp/ipykernel_29141/1946110400.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df["titulo"] = general_df["titulo"].apply(lambda x: emoji.replace_emoji(x, replace=""))
/tmp/ipykernel_29141/1946110400.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_df["oferta_activada"] = general_df["oferta_activada"].apply(lambda x: 1 if x == "Activa" else 0)
/tmp/ipykernel_29141/1946110400.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

(               id      empresa  \
 0        id_man_1      geniova   
 1        id_man_2      novicap   
 2        id_man_3    psiconnea   
 3        id_man_4          eri   
 4        id_man_5          ust   
 ...           ...          ...   
 1221  id_man_1222      product   
 1222  id_man_1223  foundernest   
 1223  id_man_1224     mindcurv   
 1224  id_man_1225     mindcurv   
 1225  id_man_1226    marketgoo   
 
                                                     url     fecha_extraccion  \
 0     https://www.getmanfred.com/ofertas-empleo/6744...  2025-02-14 11:10:38   
 1     https://www.getmanfred.com/ofertas-empleo/6677...  2025-02-14 11:10:49   
 2     https://www.getmanfred.com/ofertas-empleo/6675...  2025-02-14 11:11:21   
 3     https://www.getmanfred.com/ofertas-empleo/4193...  2025-02-14 11:11:33   
 4     https://www.getmanfred.com/ofertas-empleo/6574...  2025-02-14 11:11:45   
 ...                                                 ...                  ...   
 1221  http

In [59]:
import pandas as pd

def tecnologias_relacion (csv = "programas_matricial.csv", tecnologias="tecnologias.csv", relacion="relacion_tecnologias.csv"):
    # Ejemplo de DataFrame
    df = pd.read_csv(csv)
    
    # Lo convertimos hacia abajo para que sea mas sencillo
    df_largo = df.melt(id_vars=["id"], var_name="tecnologia", value_name="presente")
    #Si es igual a 1 lo mantenemos
    df_largo = df_largo[df_largo["presente"] == 1].drop(columns=["presente"])
    
    #Creamos un df de tecnoligas poniendole un id 
    df_tecnologias = pd.DataFrame({"tecnologia": df_largo["tecnologia"].unique()})
    df_tecnologias["tec_id"] = range(1, len(df_tecnologias) + 1)
    
    # Relacionamos los dos
    df_relacion = df_largo.merge(df_tecnologias, on="tecnologia", how="left").drop(columns=["tecnologia"])


    df_tecnologias.to_csv(tecnologias, index = False)
    df_relacion.to_csv(relacion, index = False)

    return df_tecnologias, df_relacion

In [61]:
tecnologias_relacion()

(        tecnologia  tec_id
 0           Python       1
 1              C++       2
 2        Cinema 4D       3
 3              CRM       4
 4            Canva       5
 ..             ...     ...
 360       CircleCI     361
 361     Serverless     362
 362         Zapier     363
 363  Apache Tomcat     364
 364       git-bash     365
 
 [365 rows x 2 columns],
                id  tec_id
 0        id_man_1       1
 1       id_man_11       1
 2       id_man_13       1
 3       id_man_17       1
 4       id_man_18       1
 ...           ...     ...
 5496  id_man_1043     361
 5497  id_man_1044     362
 5498  id_man_1081     363
 5499  id_man_1089     364
 5500  id_man_1089     365
 
 [5501 rows x 2 columns])

In [63]:
import pandas as pd

def habilidades_relacion (csv = "habilidades_matricial.csv", habilidades="habilidades.csv", relacion="habilidades_relacion.csv"):
    # Ejemplo de DataFrame
    df = pd.read_csv(csv)
    
    # Lo convertimos hacia abajo para que sea mas sencillo
    df_largo = df.melt(id_vars=["id"], var_name="habilidades", value_name="presente")
    #Si es igual a 1 lo mantenemos
    df_largo = df_largo[df_largo["presente"] == 1].drop(columns=["presente"])
    
    #Creamos un df de tecnoligas poniendole un id 
    df_habilidades = pd.DataFrame({"habilidades": df_largo["habilidades"].unique()})
    df_habilidades["hab_id"] = range(1, len(df_habilidades) + 1)
    
    # Relacionamos los dos
    df_relacion = df_largo.merge(df_habilidades, on="habilidades", how="left").drop(columns=["habilidades"])


    df_habilidades.to_csv(habilidades, index = False)
    df_relacion.to_csv(relacion, index = False)

    return df_habilidades, df_relacion

In [65]:
habilidades_relacion()

(                      habilidades  hab_id
 0            Pensamiento creativo       1
 1   Tolerancia a la incertidumbre       2
 2    Resistencia a la frustración       3
 3             Comunicación verbal       4
 4            Capacidad de escucha       5
 5      Capacidad de mentorización       6
 6       Capacidad de dar feedback       7
 7         Adaptabilidad al cambio       8
 8              Visión estratégica       9
 9                    Proactividad      10
 10  Capacidad de recibir feedback      11
 11              Trabajo en equipo      12
 12     Habilidades de negociación      13
 13                      Liderazgo      14
 14            Atención al detalle      15
 15         Inteligencia emocional      16
 16       Capacidad de abstracción      17
 17      Capacidad de presentación      18
 18           Comunicación escrita      19
 19          Pensamiento analítico      20
 20     Comunicación intercultural      21
 21    Autonomía en el aprendizaje      22
 22       C

In [69]:
dfg = pd.read_csv("general_limpio.csv")
df_pres = dfg[["id", "Presencial"]]  # Extraemos las columnas necesarias del DataFrame original

def split_presencial(row):
    # Verificar si "Presencial" no es NaN y contiene "OR" o "O"
    if pd.notna(row["Presencial"]) and (" OR " in row["Presencial"] or " O " in row["Presencial"]):
        ciudades = [ciudad.strip() for ciudad in row["Presencial"].replace(" O ", " OR ").split(" OR ")]
        return pd.DataFrame({
            "id": [row["id"]] * len(ciudades),
            "Presencial": ciudades
        })
    else:
        return pd.DataFrame({"id": [row["id"]], "Presencial": [row["Presencial"]]})

# Aplicar la función y concatenamos el resultado
result_df = pd.concat(df_pres.apply(split_presencial, axis=1).tolist(), ignore_index=True)
result_df.to_csv("resultado_presencial.csv", index=False)

In [71]:
result_df

,id,Presencial
0,id_man_1,MADRID
1,id_man_2,BARCELONA
2,id_man_3,MADRID
3,id_man_4,MADRID
4,id_man_5,MADRID
...,...,...
1258,id_man_1222,NaN
1259,id_man_1223,MADRID
1260,id_man_1224,MADRID
1261,id_man_1225,NaN


In [5]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

In [6]:
ciudades = pd.read_csv("/home/bosser/Documentos/PROYECTOFINAL/CSV/CSV_manfred/resultado_presencial.csv")
lista_ciudades = [i for i in ciudades["Presencial"]]
ciudades = list(set(lista_ciudades))

In [28]:
def obtener_coordenadas(ciudades, nombre_archivo="ciudades_coordenadas.csv"):
    # Inicializamos el geolocalizador
    geolocalizador = Nominatim(user_agent="geoapi")
    
    # Lista para almacenar los datos
    datos = []

    for ciudad in ciudades:
        try:
            ubicacion = geolocalizador.geocode(ciudad, timeout=10)
            if ubicacion:
                datos.append({
                    "Ciudad": ciudad,
                    "País": ubicacion.address.split(",")[-1].strip(),
                    "Latitud": ubicacion.latitude,
                    "Longitud": ubicacion.longitude
                })
            else:
                print(f"No se encontraron coordenadas para: {ciudad}")
            sleep(0.5)  # Esperar para que no nos bloquee la API
        except Exception as e:
            print(f"Error con {ciudad}: {e}")

    # Convertimos a DataFrame e ignorar filas con NaN
    df = pd.DataFrame(datos).dropna()

    # Guardampos en CSV
    df.to_csv(nombre_archivo, index=False)
    print(f"Archivo guardado")

In [30]:
obtener_coordenadas(ciudades)

Error con nan: 'float' object has no attribute 'lower'
Archivo guardado


In [6]:
ver = pd.read_csv("/home/bosser/Documentos/PROYECTOFINAL/CSV/CSV_manfred/ciudades_coordenadas.csv")

In [7]:
ver

,Ciudad,País,Latitud,Longitud
0,VALENCIA,España,39.469707,-0.376335
1,MÁLAGA,España,36.721303,-4.421637
2,LÉRIDA,España,41.614761,0.626784
3,ALCOY,España,38.698235,-0.474762
4,BARCELONA,España,41.382894,2.177432
5,VITORIA-GASTEIZ,España,42.846509,-2.672403
6,PAMPLONA,España,42.815686,-1.652235
7,ALICANTE (ALACANT),España,38.343637,-0.488171
8,SEVILLA,España,37.388630,-5.995340
9,"LISBOA, PORTUGAL",Portugal,38.707751,-9.136592
